<font color = blue><font size = 6> H(ALF)DLSTM </font></font><br>
결론적으로 LSTM은 seasonal에 주고 trend와 seasonal의 비율을 적당히 맞추는게 중요함<br>
&rightarrow; 이번에는 scaling 없이<br>
* Sample3 에 대해서

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [28]:
import torch
import torch.nn as nn
from pytorch_forecasting.metrics import SMAPE
from torch.utils.data import DataLoader,TensorDataset
from torch import FloatTensor
from torch.optim import Adam

In [29]:
use_col = ['volumn']
data = pd.read_csv('../../traffic/data/3.csv',usecols=use_col)
data.rename({'volumn':'value'},axis=1,inplace=True)
data['value'] = data['value'] * 8 / 1000000
data

,value
0,0.220168
1,0.221584
2,0.205680
3,0.185184
4,0.173464
...,...
38875,0.166704
38876,0.172936
38877,0.172976
38878,0.169264


In [30]:
import configparser
import os
config = configparser.ConfigParser()
config['online_learning_seq2seq']= {
    'input_size':1,
    'num_layers':1,
    'hidden_size':16,
}
with open('online_learning_seq2seq','w') as f:
    config.write(f)
    
config.read(os.getcwd()+os.sep+'online_learning_seq2seq')

['/hdd_1/sjlee/Paper_Project/SelfMadeModel/online_learning_seq2seq']

In [31]:
class Seasonal_Encoder(nn.Module):
    def __init__(self,configs):
        super(Seasonal_Encoder,self).__init__()
        self.input_size = int(configs['online_learning_seq2seq']['input_size'])
        # input의 feature dimension을 넣어주어야 한다
        self.hidden_size = int(configs['online_learning_seq2seq']['hidden_size'])
        # 내부에서 feature dimension을 어떻게 바꿔주고 싶은지 넣어주면 된다
        # 가령 mxn matrix가 입력으로 들어왔을때 hidden size를 h라 한다면 mxh의 크기로 바꾼다
        self.num_layers = int(configs['online_learning_seq2seq']['num_layers'])
        self.lstm = nn.LSTM(input_size=self.input_size,hidden_size = self.hidden_size,
                           num_layers = self.num_layers,batch_first = True)

    
    def forward(self,x):
        lstm_out,hidden = self.lstm(x) 
        #lstm의 output으로 나오는 hidden_state는 마지막 hidden_state값이다
        # hidden에는 튜플형태로 hidden_state와 cell_state가 둘다 포함돼있다
        return lstm_out,hidden

In [32]:
class Seasonal_Decoder(nn.Module):
    def __init__(self,configs):
        super(Seasonal_Decoder,self).__init__()
        self.input_size = int(configs['online_learning_seq2seq']['input_size'])
        # input의 feature dimension을 넣어주어야 한다
        self.hidden_size = int(configs['online_learning_seq2seq']['hidden_size'])
        # 내부에서 feature dimension을 어떻게 바꿔주고 싶은지 넣어주면 된다
        # 가령 mxn matrix가 입력으로 들어왔을때 hidden size를 h라 한다면 mxh의 크기로 바꾼다
        self.num_layers = int(configs['online_learning_seq2seq']['num_layers'])
        self.lstm = nn.LSTM(input_size=self.input_size,hidden_size = self.hidden_size,
                           num_layers = self.num_layers,batch_first = True)
        self.linear = nn.Linear(self.hidden_size,self.input_size)
        
    def forward(self,x,encoder_hidden):

        lstm_out,_ = self.lstm(x,encoder_hidden)
        
        output = self.linear(lstm_out)

        return output,_
# 정리하자면 우리가 정답을 알고 있는 시점에서 1시점 전까지의 데이터를 encoder로 넣고
# 알고 있는 마지막시점 정보와 encoder에 넣어서 나온 hidden state를 넣어줌으로써 다음시점을 예측

In [33]:
tup= (1,2)
len(tup)

2

In [34]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        
        x = x.permute(0, 2, 1)
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)

        x = torch.cat([front, x, end], dim=1)
       
        x = self.avg(x.permute(0,2,1))

        return x

In [35]:
class series_decomp(nn.Module):
    "statsmodels.tsa의 seasonal_decompose와 역할이 똑같다"
    def __init__(self,kernel_size):
        super(series_decomp,self).__init__()
        self.moving_avg = moving_avg(kernel_size,stride = 1)
    
    def forward(self,x):
        moving_mean = self.moving_avg(x) #output은 (batch,sequence_length,input_size)
        res = x - moving_mean 
        # Classical ma중에서 additive model

        return res,moving_mean

In [36]:
class seq2seq(nn.Module):
    def __init__(self,configs):
        super(seq2seq,self).__init__()
        self.device = torch.device('cuda:0')
        self.season_encoder = Seasonal_Encoder(configs)
        self.season_decoder = Seasonal_Decoder(configs)
        
        self.Linear_Trend = nn.Linear(1440,1)
        
        kernel_size = 25
        self.decomposition = series_decomp(kernel_size)
        
    def forward(self,x):
        outputs = torch.zeros(x.size(0),1,1) # feature가1개고 target도 1개인 데이터
        x = x.permute(0,2,1)
        seasonal_init,trend_init = self.decomposition(x) # res = seasonality 를 moving_mean은 trend를 의미함
        seasonal_init,trend_init = seasonal_init.permute(0,2,1).to(self.device), trend_init.to(self.device) 

        _,final_season_hidden = self.season_encoder(seasonal_init)
        season_output,_ = self.season_decoder(x = seasonal_init[:,-1,:].view(-1,1,1),encoder_hidden = final_season_hidden)
        trend_output = self.Linear_Trend(trend_init)
        
        output = trend_output+season_output
        
        return output,season_output,trend_output

In [37]:
def seq_data(x,sequence_length,pred_len):
    seq_list = []
    target_list = []
    device = torch.device("cuda:0")
    if (type(x)==list)|(type(x)==np.ndarray):
        for i in range(len(x)-sequence_length):
            seq_list.append(x[i:i+sequence_length].values)
            target_list.append(x[i+sequence_length])
    
    if (type(x) == pd.Series)|(type(x) == pd.DataFrame):
        for i in range(len(x)-sequence_length):
            seq_list.append(x.iloc[i:i+sequence_length].values)
            target_list.append(x.iloc[i+sequence_length])

#             if i == 0:
#                  print(x.iloc[i:i+sequence_length].values)
#                  print(y[i+sequence_length:i+sequence_length+pred_len].values)
            #print(x[i+sequence_length:i+sequence_length+pred_len].values.shape)
    else:
        print('error')

    return FloatTensor(seq_list).view(-1,1,sequence_length).to(device),FloatTensor(target_list).unsqueeze(1).view(-1,1,pred_len).to(device)

In [38]:
sequence_length = 1440
pred_len = 1
split= 10080
x_data = pd.DataFrame({'value':data.iloc[:-split].values.reshape(-1)})
x_seq,target = seq_data(x=x_data,sequence_length=sequence_length,pred_len=pred_len)

In [39]:
x_seq

tensor([[[0.2202, 0.2216, 0.2057,  ..., 0.2027, 0.1702, 0.1716]],

        [[0.2216, 0.2057, 0.1852,  ..., 0.1702, 0.1716, 0.2092]],

        [[0.2057, 0.1852, 0.1735,  ..., 0.1716, 0.2092, 0.2202]],

        ...,

        [[0.1641, 0.1660, 0.1776,  ..., 0.1784, 0.1696, 0.1720]],

        [[0.1660, 0.1776, 0.2252,  ..., 0.1696, 0.1720, 0.1768]],

        [[0.1776, 0.2252, 0.2034,  ..., 0.1720, 0.1768, 0.1615]]],
       device='cuda:0')

In [40]:
target

tensor([[[0.2092]],

        [[0.2202]],

        [[0.2011]],

        ...,

        [[0.1768]],

        [[0.1615]],

        [[0.1736]]], device='cuda:0')

In [41]:
import torch.optim as optim

device = torch.device('cuda:0')
batch_size = 32
num_epochs = 100
train_dataset = TensorDataset(x_seq,target)
training = DataLoader(train_dataset,batch_size=batch_size)
model = seq2seq(config).to(device)
optimizer = Adam(model.parameters(),6*1e-7)
criterion = SMAPE()
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,verbose = True)
                                       # epoch마다 learning rate에 곱해지는 값이 제곱수로 늘어남

Adjusting learning rate of group 0 to 6.0000e-07.


In [42]:
torch.manual_seed(42)

In [43]:
x_seq

tensor([[[0.2202, 0.2216, 0.2057,  ..., 0.2027, 0.1702, 0.1716]],

        [[0.2216, 0.2057, 0.1852,  ..., 0.1702, 0.1716, 0.2092]],

        [[0.2057, 0.1852, 0.1735,  ..., 0.1716, 0.2092, 0.2202]],

        ...,

        [[0.1641, 0.1660, 0.1776,  ..., 0.1784, 0.1696, 0.1720]],

        [[0.1660, 0.1776, 0.2252,  ..., 0.1696, 0.1720, 0.1768]],

        [[0.1776, 0.2252, 0.2034,  ..., 0.1720, 0.1768, 0.1615]]],
       device='cuda:0')

In [44]:
out_list = []
loss_list = []
ss_list = []
tr_list = []
n = len(training)
for i in range(num_epochs):
    running_loss = 0.0
    for seq,tg in training:
        
        seq = seq.permute(0,2,1)
#         print(seq)
        out,ss,tr = model(seq)
#        print(out)
        loss = criterion(out,tg)
    
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step() # optimizer 최적화
        
        ss_list.append(ss)
        tr_list.append(tr)
        
        running_loss = running_loss + loss.item()        
    scheduler.step() #schedular update 
    loss_list.append(running_loss/n)
    print('smape',running_loss/n*100,'%')                

Adjusting learning rate of group 0 to 5.7000e-07.
smape 16.87937249565682 %
Adjusting learning rate of group 0 to 5.4150e-07.
smape 17.05238528418959 %
Adjusting learning rate of group 0 to 5.1442e-07.
smape 17.07327510704074 %
Adjusting learning rate of group 0 to 4.8870e-07.
smape 17.090774301809873 %
Adjusting learning rate of group 0 to 4.6427e-07.
smape 17.103639875064815 %
Adjusting learning rate of group 0 to 4.4106e-07.
smape 17.119615631755334 %
Adjusting learning rate of group 0 to 4.1900e-07.
smape 17.12907920934652 %
Adjusting learning rate of group 0 to 3.9805e-07.
smape 17.135201584961678 %
Adjusting learning rate of group 0 to 3.7815e-07.
smape 17.13865364678422 %
Adjusting learning rate of group 0 to 3.5924e-07.
smape 17.137853589497116 %
Adjusting learning rate of group 0 to 3.4128e-07.
smape 17.139928312155238 %
Adjusting learning rate of group 0 to 3.2422e-07.
smape 17.141180365667704 %
Adjusting learning rate of group 0 to 3.0801e-07.
smape 17.138317531050994 %
Adju

In [45]:
print(ss_list[26])
print(tr_list[0])

tensor([[[0.1959]],

        [[0.1959]],

        [[0.1959]],

        [[0.1958]],

        [[0.1959]],

        [[0.1960]],

        [[0.1962]],

        [[0.1959]],

        [[0.1958]],

        [[0.1960]],

        [[0.1962]],

        [[0.1961]],

        [[0.1960]],

        [[0.1958]],

        [[0.1958]],

        [[0.1960]],

        [[0.1961]],

        [[0.1958]],

        [[0.1957]],

        [[0.1959]],

        [[0.1960]],

        [[0.1958]],

        [[0.1959]],

        [[0.1958]],

        [[0.1959]],

        [[0.1960]],

        [[0.1961]],

        [[0.1958]],

        [[0.1958]],

        [[0.1959]],

        [[0.1960]],

        [[0.1960]]], device='cuda:0', grad_fn=<ViewBackward0>)
tensor([[[-0.0188]],

        [[-0.0197]],

        [[-0.0193]],

        [[-0.0180]],

        [[-0.0171]],

        [[-0.0167]],

        [[-0.0171]],

        [[-0.0180]],

        [[-0.0168]],

        [[-0.0160]],

        [[-0.0158]],

        [[-0.0167]],

        [[-0.0161]],



In [ ]:
device = torch.device('cuda:0')
X = data.iloc[:-split]
target = data.iloc[-split:]
pred_list = []
time_list = []

for idx in range(split):
    running_loss = 0.0
    start = time.time()
    model.train()
    for epochs in range(num_epochs):
        if idx == 0: 
            new_train = torch.FloatTensor(
                [X.iloc[-sequence_length-1:-1].values] # 현재 알고 있는 마지막 정답에서 하나 이전 값 까지가 Training Data
            ).view(1,-1,1).to(device)

            tg = FloatTensor(X.iloc[-1]).view(1,1,1).to(device)
           
        elif idx == 1:
            new_train = torch.FloatTensor(
                [X.iloc[-sequence_length-1+idx:].values] # 현재 알고 있는 마지막 정답에서 하나 이전 값 까지가 Training Data
            ).view(1,-1,1).to(device)
            
            tg = FloatTensor(target.iloc[0]).view(1,1,1).to(device)
        else:        
            if idx <= sequence_length:# idx가 1보다는 크고 sequence_length보단 작거나 같은경우
                new_train = torch.FloatTensor(
                    [pd.concat([X.iloc[-sequence_length+idx-1:],target.iloc[0:idx-1]],axis=0,ignore_index=True).values]
                    ).view(1,-1,1).to(device)

                tg = FloatTensor(target.iloc[idx-1]).view(1,1,1).to(device)

            elif idx > sequence_length:
                new_train = torch.FloatTensor(
                    [target.iloc[idx-sequence_length-1:idx-1].values]
                    ).view(1,-1,1).to(device)
                
                tg = FloatTensor(target.iloc[idx-1]).view(1,1,1).to(device)

        out,_,_ = model(new_train) # 예측완료

        loss = criterion(out,tg)# 예측값과 타겟값의 비교

        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step() # optimizer 최적화
            
        running_loss = running_loss + loss.item()
    
    real_input = torch.cat([new_train[:,1:,:],tg],dim=1)
    
    with torch.no_grad(): #model.eval()과 train()은 레이어에만 영향을 준다 with문이 끝나면 자동 종료
        model.eval()
        real_out,_,_ = model(real_input)
    model.train()        
    print('SMAPE',(running_loss/100)*100,'%')
    end = time.time()
    pred_list.append(real_out.cpu().view(1).item())
    time_list.append(end-start)


SMAPE 0.9175995877012612 %
SMAPE 21.916460812091827 %
SMAPE 11.184759311378002 %
SMAPE 4.321214824914932 %
SMAPE 6.16332459077239 %
SMAPE 7.375504039227962 %
SMAPE 2.7913573551923037 %
SMAPE 3.605520438402891 %
SMAPE 3.9901476353406906 %
SMAPE 0.9488468365743756 %
SMAPE 0.757023150101304 %
SMAPE 16.391587629914284 %
SMAPE 4.811408903449774 %
SMAPE 2.4884577598422766 %
SMAPE 9.94059856981039 %
SMAPE 1.3822094984352589 %
SMAPE 12.422414802014828 %
SMAPE 14.193606808781622 %
SMAPE 10.230474852025509 %
SMAPE 8.793516799807549 %
SMAPE 15.223452165722847 %
SMAPE 5.714410055428743 %
SMAPE 9.687737688422203 %
SMAPE 13.074432656168938 %
SMAPE 10.496531456708908 %
SMAPE 2.4041132908314466 %
SMAPE 18.055969446897507 %
SMAPE 12.344357118010521 %
SMAPE 13.376409783959389 %
SMAPE 17.296808153390884 %
SMAPE 18.19660075008869 %
SMAPE 5.634334094822407 %
SMAPE 8.602675512433052 %
SMAPE 10.50644613802433 %
SMAPE 7.1310103088617325 %
SMAPE 2.7246586978435516 %
SMAPE 17.794057369232178 %
SMAPE 14.33617873

SMAPE 8.873192884027958 %
SMAPE 7.71059539169073 %
SMAPE 1.0900942329317331 %
SMAPE 0.4104932760819793 %
SMAPE 16.79434260725975 %
SMAPE 2.4062265548855066 %
SMAPE 16.724928110837936 %
SMAPE 4.932546369731426 %
SMAPE 4.321420673280954 %
SMAPE 8.898174092173576 %
SMAPE 10.29226503521204 %
SMAPE 8.132037051022053 %
SMAPE 0.2986423526890576 %
SMAPE 4.582689177244902 %
SMAPE 3.1259546987712383 %
SMAPE 2.895648181438446 %
SMAPE 11.604893326759338 %
SMAPE 1.4349519945681095 %
SMAPE 11.111882776021957 %
SMAPE 12.055960722267628 %
SMAPE 9.762230336666107 %
SMAPE 19.74695335328579 %
SMAPE 9.092806063592434 %
SMAPE 1.8899239785969257 %
SMAPE 6.277207605540752 %
SMAPE 5.241155814379454 %
SMAPE 5.934329450130463 %
SMAPE 0.18093301297631115 %
SMAPE 1.5624412083998322 %
SMAPE 5.343043506145477 %
SMAPE 4.641774743795395 %
SMAPE 14.359708309173582 %
SMAPE 3.5906217806041236 %
SMAPE 4.348971616476774 %
SMAPE 5.375432409346104 %
SMAPE 3.748437538743019 %
SMAPE 12.672320127487183 %
SMAPE 12.9400183260440

SMAPE 7.884079717099667 %
SMAPE 7.297482527792454 %
SMAPE 0.21410191536415366 %
SMAPE 5.980756290256977 %
SMAPE 4.736456785351038 %
SMAPE 13.280216231942177 %
SMAPE 6.056965563446283 %
SMAPE 3.4311387464404106 %
SMAPE 4.181174628436565 %
SMAPE 1.1059081004932523 %
SMAPE 4.4260781444609165 %
SMAPE 20.08947755396366 %
SMAPE 14.18327757716179 %
SMAPE 5.675448976457119 %
SMAPE 1.0400279648602009 %
SMAPE 18.143385723233223 %
SMAPE 1.0255658607929945 %
SMAPE 2.99635892175138 %
SMAPE 5.743811368942261 %
SMAPE 1.7227435819804668 %
SMAPE 3.554410245269537 %
SMAPE 9.421516485512257 %
SMAPE 7.786164082586764 %
SMAPE 0.023362124079994828 %
SMAPE 1.1972314873710275 %
SMAPE 14.092338219285011 %
SMAPE 7.147264309227467 %
SMAPE 4.2888905219733715 %
SMAPE 0.27331214281730354 %
SMAPE 3.9707053750753403 %
SMAPE 3.918096210807562 %
SMAPE 10.547883123159409 %
SMAPE 10.944755606353283 %
SMAPE 5.036832723766565 %
SMAPE 8.273405216634274 %
SMAPE 8.243277452886105 %
SMAPE 11.038236573338509 %
SMAPE 8.329732261

SMAPE 9.895924232900143 %
SMAPE 4.063594784587622 %
SMAPE 1.0916754510253668 %
SMAPE 7.921470068395138 %
SMAPE 10.029126718640327 %
SMAPE 11.191679492592812 %
SMAPE 0.033657326974207535 %
SMAPE 5.97296940907836 %
SMAPE 8.035441182553768 %
SMAPE 8.229089118540287 %
SMAPE 3.964624878019094 %
SMAPE 2.4608039278537035 %
SMAPE 0.32679488277062774 %
SMAPE 4.4707573018968105 %
SMAPE 6.87165454775095 %
SMAPE 7.657215312123299 %
SMAPE 6.113945223391056 %
SMAPE 6.182758167386055 %
SMAPE 10.706310234963894 %
SMAPE 10.732092745602131 %
SMAPE 6.5622986778616905 %
SMAPE 3.321611683815718 %
SMAPE 0.9788307417184114 %
SMAPE 3.824880752712488 %
SMAPE 11.888843610882759 %
SMAPE 7.491528138518333 %
SMAPE 4.344288185238838 %
SMAPE 1.7459413781762123 %
SMAPE 6.413045160472393 %
SMAPE 5.938854496926069 %
SMAPE 19.86236445605755 %
SMAPE 14.459016621112825 %
SMAPE 8.219554737210274 %
SMAPE 4.277282498776913 %
SMAPE 14.237828508019449 %
SMAPE 8.869529969990253 %
SMAPE 5.578771065920591 %
SMAPE 2.09001272916793

SMAPE 6.632168665528297 %
SMAPE 7.914219118654728 %
SMAPE 0.783156213350594 %
SMAPE 0.18025114678312093 %
SMAPE 4.831146158277988 %
SMAPE 15.023342579603193 %
SMAPE 9.224418759346008 %
SMAPE 1.8012809511274102 %
SMAPE 1.171607268974185 %
SMAPE 2.0863425116986036 %
SMAPE 6.587377458810806 %
SMAPE 8.068086870014668 %
SMAPE 2.4935646038502455 %
SMAPE 0.6768211256712675 %
SMAPE 3.0289753451943398 %
SMAPE 10.962473399937153 %
SMAPE 6.936935260891914 %
SMAPE 0.8473039334639908 %
SMAPE 3.282578971236944 %
SMAPE 2.702542604878545 %
SMAPE 5.490666575729847 %
SMAPE 28.232678830623627 %
SMAPE 20.33844266831875 %
SMAPE 15.493394061923027 %
SMAPE 10.292194738984108 %
SMAPE 25.36274042725563 %
SMAPE 4.733055580407381 %
SMAPE 10.918906956911087 %
SMAPE 13.665638327598572 %
SMAPE 11.30196237564087 %
SMAPE 12.508839875459671 %
SMAPE 22.55637615919113 %
SMAPE 13.38488319516182 %
SMAPE 8.60725474357605 %
SMAPE 8.084456391632557 %
SMAPE 19.510725259780884 %
SMAPE 3.468844920396805 %
SMAPE 7.15093116462230

...

SMAPE 4.303325191140175 %
SMAPE 3.874051906168461 %
SMAPE 1.9773352108895779 %
SMAPE 0.3737208235543221 %
SMAPE 18.550216898322105 %
SMAPE 0.11633937171427533 %
SMAPE 3.6215654872357845 %
SMAPE 2.717773864045739 %
SMAPE 0.6521106283180416 %
SMAPE 16.43195715546608 %
SMAPE 10.351259358227253 %
SMAPE 9.192776910960674 %
SMAPE 1.3253041999414563 %
SMAPE 5.793727476149797 %
SMAPE 25.710650473833084 %
SMAPE 6.1744323670864105 %
SMAPE 7.622758820652961 %
SMAPE 10.604982502758503 %
SMAPE 7.767353974282741 %
SMAPE 14.159804284572603 %
SMAPE 0.3071317875292152 %
SMAPE 2.678659623488784 %
SMAPE 1.254158180207014 %
SMAPE 0.40509961917996407 %
SMAPE 28.546358346939087 %
SMAPE 8.570846624672413 %
SMAPE 9.214359976351261 %
SMAPE 12.498030044138432 %
SMAPE 17.880452424287796 %
SMAPE 21.633279353380203 %
SMAPE 9.21904057264328 %
SMAPE 10.736998610198498 %
SMAPE 13.155376851558687 %
SMAPE 18.995768412947655 %
SMAPE 31.033616960048676 %
SMAPE 13.035367280244827 %
SMAPE 13.878759056329725 %
SMAPE 18.5310

SMAPE 28.16591945290565 %
SMAPE 26.162298530340195 %
SMAPE 18.320463836193085 %
SMAPE 28.206203311681747 %
SMAPE 8.998445965349674 %
SMAPE 7.3114301040768614 %
SMAPE 10.575898580253124 %
SMAPE 7.945947229862213 %
SMAPE 1.1431132601574063 %
SMAPE 4.783277221024036 %
SMAPE 11.950526542961597 %
SMAPE 12.313909538090229 %
SMAPE 7.337715983390808 %
SMAPE 8.118002384901047 %
SMAPE 12.283644907176495 %
SMAPE 3.4046306274831295 %
SMAPE 4.980777386575937 %
SMAPE 1.3122060736641288 %
SMAPE 2.9636891558766365 %
SMAPE 0.16273448267020285 %
SMAPE 13.984036028385164 %
SMAPE 7.30498556047678 %
SMAPE 1.863525502383709 %
SMAPE 4.553884971886873 %
SMAPE 15.076076552271845 %
SMAPE 3.9386049099266534 %
SMAPE 3.4772628284990788 %
SMAPE 0.5944200586527586 %
SMAPE 4.928592503070831 %
SMAPE 8.552458219230175 %
SMAPE 14.154552400112152 %
SMAPE 7.59648372232914 %
SMAPE 3.589371703565121 %
SMAPE 4.3582501001656055 %
SMAPE 18.41136883199215 %
SMAPE 0.9684952488169075 %
SMAPE 0.286666622152552 %
SMAPE 4.2765083611

SMAPE 5.412243530154228 %
SMAPE 2.819622155278921 %
SMAPE 8.88081119209528 %
SMAPE 17.90693572163582 %
SMAPE 4.102811753749847 %
SMAPE 7.054108485579491 %
SMAPE 18.98481434583664 %
SMAPE 28.165601909160614 %
SMAPE 0.45079027628526097 %
SMAPE 5.395075485110283 %
SMAPE 15.461147397756578 %
SMAPE 0.8835535682737826 %
SMAPE 3.845463652163744 %
SMAPE 13.630948409438135 %
SMAPE 1.5778264962136745 %
SMAPE 5.545535512268543 %
SMAPE 4.397264454513788 %
SMAPE 0.407915843417868 %
SMAPE 2.6824907399713993 %
SMAPE 11.01516679674387 %
SMAPE 10.288071289658546 %
SMAPE 4.657714605331421 %
SMAPE 0.1412367463344708 %
SMAPE 13.21545870602131 %
SMAPE 4.319349933415651 %
SMAPE 7.102769173681736 %
SMAPE 0.6953749507665634 %
SMAPE 2.7961150258779526 %
SMAPE 7.692610293626785 %
SMAPE 2.8933778256177902 %
SMAPE 8.590307652950287 %
SMAPE 5.076926793903112 %
SMAPE 1.225716213695705 %
SMAPE 17.56909589469433 %
SMAPE 1.9881887417286632 %
SMAPE 3.424400005489588 %
SMAPE 6.345634289085865 %
SMAPE 7.052190870046616 %

SMAPE 6.538102455437183 %
SMAPE 0.9736521486192942 %
SMAPE 10.031642034649849 %
SMAPE 4.935790907591581 %
SMAPE 11.580278538167477 %
SMAPE 1.438370369374752 %
SMAPE 3.0785091295838356 %
SMAPE 5.251404833048582 %
SMAPE 10.362075977027416 %
SMAPE 9.424978218972683 %
SMAPE 9.68648425489664 %
SMAPE 2.244393842294812 %
SMAPE 12.526287108659744 %
SMAPE 10.001244433224201 %
SMAPE 0.7445563208311796 %
SMAPE 1.373128823004663 %
SMAPE 1.4877450950443745 %
SMAPE 4.728218469768763 %
SMAPE 7.599754631519318 %
SMAPE 12.584662750363352 %
SMAPE 4.151298053562641 %
SMAPE 9.823785312473774 %
SMAPE 17.008480742573738 %
SMAPE 16.52231851220131 %
SMAPE 7.82438238710165 %
SMAPE 1.4204125935211778 %
SMAPE 10.412970311939716 %
SMAPE 15.143667414784431 %
SMAPE 0.556917006149888 %
SMAPE 0.4282787223346531 %
SMAPE 2.354465549811721 %
SMAPE 14.502248331904411 %
SMAPE 23.65523624420166 %
SMAPE 18.80560564994812 %
SMAPE 10.755568914115429 %
SMAPE 12.999171465635301 %
SMAPE 11.868433982133865 %
SMAPE 5.3878504075109

SMAPE 28.705289781093597 %
SMAPE 7.819470040500163 %
SMAPE 4.862261015921831 %
SMAPE 3.8188385404646397 %
SMAPE 7.615136742591858 %
SMAPE 3.511569745838642 %
SMAPE 4.704829819500446 %
SMAPE 0.0025739185644368945 %
SMAPE 11.554325751960278 %
SMAPE 16.75590306520462 %
SMAPE 28.13437110185623 %
SMAPE 14.699652954936028 %
SMAPE 0.3110908621456474 %
SMAPE 4.336373779922724 %
SMAPE 1.8195605389773846 %
SMAPE 7.630756825208664 %
SMAPE 3.0965015590190887 %
SMAPE 4.7108086831867695 %
SMAPE 16.441411957144737 %
SMAPE 16.721372827887535 %
SMAPE 32.03245159983635 %
SMAPE 11.91904603689909 %
SMAPE 9.791849486529827 %
SMAPE 3.802167806774378 %
SMAPE 6.436454817652702 %
SMAPE 13.356321021914482 %
SMAPE 8.512992933392525 %
SMAPE 4.9635042399168015 %
SMAPE 24.447475269436836 %
SMAPE 19.52159921824932 %
SMAPE 32.926675885915756 %
SMAPE 16.60476705431938 %
SMAPE 2.0131679121404886 %
SMAPE 0.3801206639036536 %
SMAPE 2.832744313403964 %
SMAPE 4.996859282255173 %
SMAPE 3.1888755410909653 %
SMAPE 0.637672733

In [ ]:
len(pred_list)

In [ ]:
target=list(target.values.reshape(-1))

In [ ]:
len(target)

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(12,8))
axes.plot(np.arange(1,61),pred_list[:60],label='prediction')
axes.plot(np.arange(1,61),target[:60],label='target')
axes.legend()

In [ ]:
target = data.iloc[-split:]
target = target.values.reshape(-1,1)
score=SMAPE()(FloatTensor(np.array(pred_list).reshape(-1,1)),FloatTensor(target))*100
print('SMAPE',score.item(),'%')

In [ ]:
pred_list

In [ ]:
len(target)

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(12,8))
axes.plot(np.arange(1,10081),pred_list[:],label='prediction')
axes.plot(np.arange(1,10081),target[:],label='target')
axes.legend()

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(12,8))
axes.plot(np.arange(1,61),pred_list[-60:],label='prediction')
axes.plot(np.arange(1,61),target[-60:],label='target')
axes.legend()

In [ ]:
np.mean(time_list)

1step과 비교했을때level을 맞추면서 sliding을 하게됨

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(12,8))
axes.plot(np.arange(2200,2250),pred_list[2200:2250],label='prediction')
axes.plot(np.arange(2200,2250),target[2200:2250],label='target')
axes.legend()